# BLIP-2 LoRA Fine-Tuning on UICD

Fine-tune **BLIP-2 (Salesforce/blip2-opt-2.7b)** with **LoRA** on the **Underwater Image Captioning Dataset (UICD)**.

| Setting | Value |
|---|---|
| Base model | `Salesforce/blip2-opt-2.7b` |
| Precision | `fp16` on GPU, `fp32` on CPU (no quantization) |
| LoRA rank `r` | 16 |
| LoRA alpha | 16 |
| Target modules | `q_proj`, `k_proj` |
| Dropout | 0.1 |
| Bias | `all` |
| Trainable params | ~0.77% |
| Dataset split | 80 / 10 / 10 (train / val / test) |
| Captions per image | 5 (all captions used) |
| GPU | NVIDIA GeForce RTX 3090 Ti (or CPU fallback) |


## 0. Environment Setup (Colab)

Install required packages if running on Google Colab.

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# COLAB SETUP — Install packages not in default Colab environment
# ══════════════════════════════════════════════════════════════════════════════

import sys

# Detect if running on Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Running on Google Colab — installing required packages...\n")

    # Install PEFT (Parameter-Efficient Fine-Tuning) for LoRA
    print("Installing peft...")
    !pip install -q peft

    # Install pycocoevalcap for caption evaluation metrics (BLEU, CIDEr, METEOR, ROUGE)
    print("Installing pycocoevalcap...")
    !pip install -q pycocoevalcap

    # Install tensorboard for training metrics logging
    print("Installing tensorboard...")
    !pip install -q tensorboard

    # Ensure latest transformers is installed
    print("Upgrading transformers...")
    !pip install -q --upgrade transformers

    print("\n✓ All packages installed successfully!")
else:
    print("ℹ️  Not running on Colab — skipping package installation.")
    print("   Make sure you have: peft, pycocoevalcap, tensorboard, transformers")


🔧 Running on Google Colab — installing required packages...

Installing peft...
Installing pycocoevalcap...
Upgrading transformers...

✓ All packages installed successfully!


## 1. Imports & Environment

In [ ]:
import os
import re
import json
import random
import copy
from pathlib import Path
from collections import defaultdict

import torch
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

# Hugging Face
from transformers import AutoProcessor, AutoModelForCausalLM, TrainingArguments, Trainer, TrainerCallback
from transformers import Blip2ForConditionalGeneration

# PEFT / LoRA
from peft import LoraConfig, get_peft_model, TaskType

# Evaluation
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge

# ── Check TensorBoard availability ───────────────────────────────────────────
try:
    import tensorboard
    HAS_TENSORBOARD = True
except ImportError:
    HAS_TENSORBOARD = False
    print("⚠️  TensorBoard not installed — logging will be disabled")
    print("   Install with: pip install tensorboard")

# ── Device setup with CPU fallback ───────────────────────────────────────────
if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb  = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"🚀 GPU  : {gpu_name}")
    print(f"   VRAM : {vram_gb:.1f} GB")
    # Use float16 for GPU efficiency
    model_dtype = torch.float16
    use_pin_memory = True
    use_fp16 = True
else:
    device = torch.device("cpu")
    print(f"⚠️  CUDA not available — using CPU")
    print(f"   (Training will be significantly slower)")
    # Use float32 on CPU (float16 is very slow on CPU)
    model_dtype = torch.float32
    use_pin_memory = False
    use_fp16 = False  # Disable fp16 on CPU to avoid accelerate issues

print(f"PyTorch: {torch.__version__}")
print(f"Device : {device}")
print(f"dtype  : {model_dtype}")
print(f"fp16   : {use_fp16}")


GPU : Tesla T4
VRAM: 14.6 GB
PyTorch: 2.10.0+cu128


## 2. Extract Dataset

Unzip `uic.zip` into `data/uic/` if it hasn't been extracted yet.

In [3]:
import zipfile
import shutil

zip_file_path  = "uic.zip"
extract_to_path = "data/uic"

if os.path.exists(extract_to_path):
    print(f"Removing existing {extract_to_path}...")
    shutil.rmtree(extract_to_path)

# Extract zip file if it exists
if os.path.exists(zip_file_path):
    os.makedirs(extract_to_path, exist_ok=True)
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"Successfully extracted {zip_file_path} to {extract_to_path}")
else:
    print(f"'{zip_file_path}' not found — assuming data is already extracted at {extract_to_path}/")

Removing existing data/uic...
Successfully extracted uic.zip to data/uic


## 2. Load Base Model & Processor

Load `Salesforce/blip2-opt-2.7b` in **fp16** (GPU) or **fp32** (CPU) without quantization so that LoRA low-rank matrices receive clean gradients during backpropagation. All base model parameters are frozen immediately after loading.

In [ ]:
MODEL_ID = "Salesforce/blip2-opt-2.7b"

# ── Processor ────────────────────────────────────────────────────────────────
processor = AutoProcessor.from_pretrained(MODEL_ID)
print("Processor loaded.")

# ── Base model (with device-appropriate dtype) ───────────────────────────────
base_model = Blip2ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=model_dtype,
).to(device)
print("Base model loaded.")

# ── Freeze ALL base model parameters ─────────────────────────────────────────
for param in base_model.parameters():
    param.requires_grad = False

total_params = sum(p.numel() for p in base_model.parameters())
print(f"Total parameters (all frozen): {total_params:,}")
print(f"Trainable parameters (pre-LoRA): {sum(p.numel() for p in base_model.parameters() if p.requires_grad):,}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The image processor of type `BlipImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


Processor loaded.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/1247 [00:00<?, ?it/s]

Base model loaded.
Total parameters (all frozen): 3,744,761,856
Trainable parameters (pre-LoRA): 0


## 3. LoRA Configuration & Injection

Inject LoRA adapters into the **query** (`q_proj`) and **key** (`k_proj`) projection layers of the OPT language model decoder — exactly as specified. `bias="all"` trains all bias vectors (including those in non-LoRA layers), which slightly improves convergence. After `get_peft_model`, only LoRA parameters are trainable.

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj"],   # attention query + key projections
    lora_dropout=0.1,
    bias="all",
    task_type=TaskType.CAUSAL_LM,           # OPT is a decoder-only causal LM
)

model = get_peft_model(base_model, lora_config)

# ── Cast LoRA (trainable) parameters to float32 ──────────────────────────────
# When using fp16, PyTorch's AMP grad scaler requires trainable parameters to
# be in fp32 so it can safely unscale gradients. On CPU, all params are fp32.
for name, param in model.named_parameters():
    if param.requires_grad:
        param.data = param.data.to(torch.float32)

# ── Trainable parameter report ────────────────────────────────────────────────
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
pct       = 100.0 * trainable / total

print(f"Trainable parameters : {trainable:>12,}")
print(f"Total parameters     : {total:>12,}")
print(f"Trainable %          : {pct:.4f}%")

if not (0.5 <= pct <= 1.5):
    model.print_trainable_parameters()
    print(f"\n⚠  Trainable % ({pct:.4f}%) is outside the expected ~0.77% range.")
    print("   Consider adding 'v_proj' to target_modules or adjusting rank `r`.")
else:
    print(f"\n✓  Parameter efficiency target met ({pct:.4f}% trainable).")

Trainable parameters :    6,901,120
Total parameters     : 3,750,004,736
Trainable %          : 0.1840%
trainable params: 6,901,120 || all params: 3,750,004,736 || trainable%: 0.1840

⚠  Trainable % (0.1840%) is outside the expected ~0.77% range.
   Consider adding 'v_proj' to target_modules or adjusting rank `r`.


## 4. Dataset Preparation (UICD)

Parse `UIC-captions.txt` (format: `uic_img_N.jpg#K caption text`), group all 5 captions per image, and perform an **80/10/10 deterministic split** at the image level (seed=42) — mirroring `blip2_explorer_experiments.ipynb` for fair comparison with the zero-shot baseline.

Initially all 5 captions per image are expanded into individual `(image_path, caption)` training records. **Note**: The dataset will be rebuilt in section 9 based on the selected training config (TESTING uses only 1 caption/image for speed).

In [6]:
# ── Paths ─────────────────────────────────────────────────────────────────────
DATA_ROOT   = Path("data/uic/UIC(underwater image captioning dataset)")
CAPTIONS_FILE = DATA_ROOT / "UIC-captions.txt"
IMAGE_DIR     = DATA_ROOT / "uic_224x224_image"

# ── Parse captions file ───────────────────────────────────────────────────────
def load_captions(captions_path: Path) -> dict:
    """Return {filename: [cap1, cap2, cap3, cap4, cap5]}."""
    captions = defaultdict(list)
    with open(captions_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Format: "uic_img_N.jpg#K caption text"
            first_space = line.index(" ")
            img_tag = line[:first_space]            # e.g. "uic_img_1.jpg#1"
            caption = line[first_space + 1:]
            filename = re.sub(r"#\d+$", "", img_tag)  # strip "#K"
            captions[filename].append(caption)
    return dict(captions)

captions_dict = load_captions(CAPTIONS_FILE)

# ── Match captions to existing image files ────────────────────────────────────
available_images = {p.name: p for p in IMAGE_DIR.glob("*.jpg")}
image_keys = sorted([k for k in captions_dict if k in available_images])

print(f"Total images with captions: {len(image_keys)}")
print(f"Sample: {image_keys[:5]}")

# ── 80 / 10 / 10 deterministic split ─────────────────────────────────────────
rng = random.Random(42)
shuffled = image_keys.copy()
rng.shuffle(shuffled)

n_total = len(shuffled)
n_train = int(0.80 * n_total)
n_val   = int(0.10 * n_total)

train_keys = shuffled[:n_train]
val_keys   = shuffled[n_train : n_train + n_val]
test_keys  = shuffled[n_train + n_val :]

print(f"\nSplit (images) — train: {len(train_keys)}, val: {len(val_keys)}, test: {len(test_keys)}")

# ── Expand to (image_path, caption) pairs — initially using ALL captions ──────
# This will be rebuilt in section 9 based on training config
def expand_records(keys, captions_dict, available_images, all_caps=True):
    """Return a list of (image_path, caption) tuples."""
    records = []
    for key in keys:
        img_path = available_images[key]
        caps = captions_dict[key]
        if all_caps:
            for cap in caps:
                records.append((img_path, cap))
        else:
            records.append((img_path, caps[0]))
    return records

train_records = expand_records(train_keys, captions_dict, available_images, all_caps=True)
val_records   = expand_records(val_keys,   captions_dict, available_images, all_caps=True)
test_records  = expand_records(test_keys,  captions_dict, available_images, all_caps=True)

# For caption generation evaluation we use one record per image (all 5 refs)
def build_eval_records(keys, captions_dict, available_images):
    """Return list of (image_path, [cap1..cap5]) for generation metrics."""
    return [(available_images[k], captions_dict[k]) for k in keys]

val_eval_records  = build_eval_records(val_keys,  captions_dict, available_images)
test_eval_records = build_eval_records(test_keys, captions_dict, available_images)

print(f"\nExpanded records — train: {len(train_records)}, val: {len(val_records)}, test: {len(test_records)}")
print(f"Eval records (1 per image) — val: {len(val_eval_records)}, test: {len(test_eval_records)}")

Total images with captions: 3176
Sample: ['uic_img_1.jpg', 'uic_img_10.jpg', 'uic_img_100.jpg', 'uic_img_1000.jpg', 'uic_img_1001.jpg']

Split (images) — train: 2540, val: 317, test: 319

Expanded records — train: 12697, val: 1585, test: 1594
Eval records (1 per image) — val: 317, test: 319


## 5. UICDataset

A `torch.utils.data.Dataset` that loads each `(image_path, caption)` record, runs it through the BLIP-2 processor, and masks padding tokens in the labels so cross-entropy loss ignores them.

In [ ]:
import torch
from torch.utils.data import Dataset

MAX_TEXT_LEN = 128   # max tokenized caption length (shared constant)

class UICDataset(Dataset):
    """
    Pairs (image_path, caption) → BLIP-2 processor inputs.

    Returns a dict with:
        pixel_values   : (3, H, W) tensor (dtype matches model_dtype)
        input_ids      : (MAX_TEXT_LEN,) long tensor
        attention_mask : (MAX_TEXT_LEN,) long tensor
        labels         : (MAX_TEXT_LEN,) long tensor  — padding masked to -100
    """

    def __init__(self, records, processor, max_length=MAX_TEXT_LEN):
        self.records    = records         # list of (Path, str)
        self.processor  = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        img_path, caption = self.records[idx]

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Process image + text together
        encoding = self.processor(
            images=image,
            text=caption,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        # Strip the leading batch dimension that return_tensors="pt" adds
        pixel_values   = encoding["pixel_values"].squeeze(0)      # (3, H, W)
        input_ids      = encoding["input_ids"].squeeze(0)          # (L,)
        attention_mask = encoding["attention_mask"].squeeze(0)     # (L,)

        # Labels: clone input_ids, mask padding (-100 is ignored by CrossEntropy)
        labels = input_ids.clone()
        pad_id = self.processor.tokenizer.pad_token_id
        if pad_id is not None:
            labels[labels == pad_id] = -100

        return {
            "pixel_values":   pixel_values.to(model_dtype),
            "input_ids":      input_ids,
            "attention_mask": attention_mask,
            "labels":         labels,
        }


# ── Quick sanity check ────────────────────────────────────────────────────────
_sample = UICDataset(train_records[:2], processor)
_item   = _sample[0]
print("pixel_values shape :", _item["pixel_values"].shape)
print("input_ids shape    :", _item["input_ids"].shape)
print("labels (first 20)  :", _item["labels"][:20].tolist())
print("Dataset sizes — train:", len(train_records),
      "| val:", len(val_records), "| test:", len(test_records))

pixel_values shape : torch.Size([3, 224, 224])
input_ids shape    : torch.Size([128])
labels (first 20)  : [50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265, 50265]
Dataset sizes — train: 12697 | val: 1585 | test: 1594


## 6. DataLoaders & Collate Function

The HF `Trainer` accepts `Dataset` objects directly but we also build explicit `DataLoader`s for reference. A simple `collate_fn` stacks the per-sample dicts returned by `UICDataset` into batched tensors.

**Note**: The datasets are rebuilt in section 9 based on the selected training configuration.

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE  = 4
NUM_WORKERS = 2

def collate_fn(batch):
    """Stack a list of sample dicts into a batched dict of tensors."""
    return {key: torch.stack([sample[key] for sample in batch]) for key in batch[0]}

# ── Instantiate datasets ──────────────────────────────────────────────────────
train_dataset = UICDataset(train_records, processor)
val_dataset   = UICDataset(val_records,   processor)
test_dataset  = UICDataset(test_records,  processor)

# ── DataLoaders (with device-appropriate pin_memory setting) ─────────────────
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, collate_fn=collate_fn,
                          pin_memory=use_pin_memory)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, collate_fn=collate_fn,
                          pin_memory=use_pin_memory)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, collate_fn=collate_fn,
                          pin_memory=use_pin_memory)

print(f"Train batches : {len(train_loader)}")
print(f"Val   batches : {len(val_loader)}")
print(f"Test  batches : {len(test_loader)}")

Train batches : 3175
Val   batches : 397
Test  batches : 399


## 7. Caption Generation & Metrics Utility

`evaluate_captions` generates predictions for a set of `(image_path, [ref_captions])` records, then scores them with **BLEU-1–4**, **CIDEr**, **METEOR**, and **ROUGE-L** using `pycocoevalcap` — the same evaluation stack used in `blip2_explorer_experiments.ipynb`.

The subsample size for validation evaluation is configurable (30 for TESTING, 150 for FULL) to balance speed vs. accuracy.

In [9]:
# Default subsample for evaluate_captions() — can be overridden by config
DEFAULT_EVAL_SUBSAMPLE = 30

def evaluate_captions(model, processor, eval_records, device,
                      max_new_tokens=50, num_beams=3,
                      subsample=None, seed=0):
    """
    Generate captions for eval_records and compute NLP metrics.

    Parameters
    ----------
    eval_records : list of (image_path, list[str])
        Each element is one image paired with its 5 reference captions.
    subsample : int or None
        If set, randomly sample this many records before evaluation.

    Returns
    -------
    dict with keys: Bleu_1, Bleu_2, Bleu_3, Bleu_4, CIDEr, METEOR, ROUGE_L
    """
    model.eval()

    if subsample and subsample < len(eval_records):
        rng = random.Random(seed)
        eval_records = rng.sample(eval_records, subsample)

    gts  = {}   # image_id -> [ref strings]
    res  = {}   # image_id -> [hyp string]

    with torch.no_grad():
        for img_id, (img_path, ref_caps) in enumerate(tqdm(eval_records, desc="Generating captions", leave=False)):
            image = Image.open(img_path).convert("RGB")
            inputs = processor(images=image, return_tensors="pt").to(device)

            generated_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                num_beams=num_beams,
                do_sample=False,
            )
            pred = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

            gts[img_id] = ref_caps
            res[img_id] = [pred]

    # ── Compute metrics ───────────────────────────────────────────────────────
    bleu_scorer   = Bleu(4)
    cider_scorer  = Cider()
    meteor_scorer = Meteor()
    rouge_scorer  = Rouge()

    bleu_scores, _  = bleu_scorer.compute_score(gts, res)
    cider_score, _  = cider_scorer.compute_score(gts, res)
    meteor_score, _ = meteor_scorer.compute_score(gts, res)
    rouge_score, _  = rouge_scorer.compute_score(gts, res)

    metrics = {
        "Bleu_1":  bleu_scores[0],
        "Bleu_2":  bleu_scores[1],
        "Bleu_3":  bleu_scores[2],
        "Bleu_4":  bleu_scores[3],
        "CIDEr":   cider_score,
        "METEOR":  meteor_score,
        "ROUGE_L": rouge_score,
    }
    model.train()
    return metrics


def print_metrics(metrics, label=""):
    header = f"  {'Metric':<12} {'Score':>8}"
    print(f"\n{'─'*24}")
    if label:
        print(f"  {label}")
    print(header)
    print(f"{'─'*24}")
    for k, v in metrics.items():
        print(f"  {k:<12} {v:>8.4f}")
    print(f"{'─'*24}\n")

print("evaluate_captions() ready.")

evaluate_captions() ready.


## 8. Generation Metrics Callback

`GenerationMetricsCallback` hooks into the HF `Trainer` at each `on_evaluate` event. It sub-samples the validation split, runs `evaluate_captions`, prints a table, and logs all scores back into the trainer's log history (which surfaces to TensorBoard / W&B).

In [ ]:
class GenerationMetricsCallback(TrainerCallback):
    """
    Runs caption generation + NLP metric evaluation on the validation split
    at every `on_evaluate` event during HF Trainer training.

    Logs results as scalar metrics so they appear in TensorBoard / W&B.
    """

    def __init__(self, processor, eval_records, device,
                 subsample=DEFAULT_EVAL_SUBSAMPLE, max_new_tokens=50, num_beams=3):
        self.processor      = processor
        self.eval_records   = eval_records
        self.device         = device
        self.subsample      = subsample
        self.max_new_tokens = max_new_tokens
        self.num_beams      = num_beams
        self._step_counter  = 0

    def on_evaluate(self, args, state, control, model=None, **kwargs):
        self._step_counter += 1
        print(f"\n[GenMetricsCallback] Eval #{self._step_counter} "
              f"(step {state.global_step}) — "
              f"sampling {self.subsample} images from val split …")

        # ── Temporarily cast all params to model_dtype for inference ─────────
        # Save original dtypes to restore after generation
        original_dtypes = {}
        for name, param in model.named_parameters():
            original_dtypes[name] = param.dtype
            if param.dtype != model_dtype:
                param.data = param.data.to(model_dtype)

        metrics = evaluate_captions(
            model=model,
            processor=self.processor,
            eval_records=self.eval_records,
            device=self.device,
            max_new_tokens=self.max_new_tokens,
            num_beams=self.num_beams,
            subsample=self.subsample,
            seed=state.global_step,      # different subsample each eval
        )

        # ── Restore original dtypes for continued training ────────────────────
        for name, param in model.named_parameters():
            if original_dtypes[name] != model_dtype:
                param.data = param.data.to(original_dtypes[name])

        print_metrics(metrics, label=f"Val Generation Metrics @ step {state.global_step}")

        # Log to Trainer (surfaces to TensorBoard / W&B under "eval/" prefix)
        log_dict = {f"eval/gen_{k}": v for k, v in metrics.items()}
        if hasattr(state, "log_history"):
            state.log_history.append({**log_dict, "step": state.global_step})

        return control


print("GenerationMetricsCallback defined.")


GenerationMetricsCallback defined.


## 9. Training Configuration

Define training configurations for different scenarios:
- **TESTING**: Fast iteration for debugging (1 epoch, 1 caption/image, infrequent eval, small subsample)
- **FULL**: Production training (3 epochs, 5 captions/image, regular eval, larger subsample)

In [11]:
# ══════════════════════════════════════════════════════════════════════════════
# TRAINING CONFIGURATION PROFILES
# ══════════════════════════════════════════════════════════════════════════════

CONFIGS = {
    "TESTING": {
        # Fast iteration config for debugging and code testing
        "num_train_epochs": 1,
        "eval_steps": 500,              # infrequent eval
        "save_steps": 500,
        "eval_subsample": 30,           # only 30 images for quick feedback
        "logging_steps": 50,
        "use_all_captions": False,      # only 1 caption per image (5x faster)
        "description": "Fast testing config (1 epoch, 1 caption/image, minimal eval)",
    },

    "FULL": {
        # Production training config for final results
        "num_train_epochs": 3,
        "eval_steps": 200,              # more frequent eval
        "save_steps": 200,
        "eval_subsample": 150,          # more thorough eval (150 images)
        "logging_steps": 25,
        "use_all_captions": True,       # all 5 captions per image
        "description": "Full training config (3 epochs, 5 captions/image, thorough eval)",
    },
}

# ────────────────────────────────────────────────────────────────────────────
# SELECT CONFIGURATION HERE
# ────────────────────────────────────────────────────────────────────────────
SELECTED_CONFIG = "TESTING"  # Change to "FULL" for production training
# ────────────────────────────────────────────────────────────────────────────

config = CONFIGS[SELECTED_CONFIG]
print(f"╔{'═'*78}╗")
print(f"║ Training Configuration: {SELECTED_CONFIG:<56} ║")
print(f"╠{'═'*78}╣")
print(f"║  {config['description']:<75} ║")
print(f"║  Captions/image   : {'All 5' if config['use_all_captions'] else '1 only':<60} ║")
print(f"║  Eval steps       : {config['eval_steps']:<60} ║")
print(f"║  Save steps       : {config['save_steps']:<60} ║")
print(f"║  Eval subsample   : {config['eval_subsample']:<60} ║")
print(f"║  Logging steps    : {config['logging_steps']:<60} ║")
print(f"╚{'═'*78}╝")

# ── Rebuild datasets based on config ──────────────────────────────────────────
use_all_caps = config["use_all_captions"]
train_records = expand_records(train_keys, captions_dict, available_images, all_caps=use_all_caps)
val_records   = expand_records(val_keys,   captions_dict, available_images, all_caps=use_all_caps)
test_records  = expand_records(test_keys,  captions_dict, available_images, all_caps=use_all_caps)

print(f"\nDataset rebuilt with {'all 5 captions' if use_all_caps else 'only 1 caption'} per image:")
print(f"  Train records: {len(train_records)}")
print(f"  Val records  : {len(val_records)}")
print(f"  Test records : {len(test_records)}")

# Rebuild dataset objects
train_dataset = UICDataset(train_records, processor)
val_dataset   = UICDataset(val_records,   processor)
test_dataset  = UICDataset(test_records,  processor)

╔══════════════════════════════════════════════════════════════════════════════╗
║ Training Configuration: TESTING                                                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  Fast testing config (1 epoch, 1 caption/image, minimal eval)                ║
║  Captions/image   : 1 only                                                       ║
║  Eval steps       : 500                                                          ║
║  Save steps       : 500                                                          ║
║  Eval subsample   : 30                                                           ║
║  Logging steps    : 50                                                           ║
╚══════════════════════════════════════════════════════════════════════════════╝

Dataset rebuilt with only 1 caption per image:
  Train records: 2540
  Val records  : 317
  Test records : 319


## 10. Initialize Trainer

Create the HF `Trainer` with the selected configuration:
- `per_device_train_batch_size=4` × `gradient_accumulation_steps=4` → **effective batch 16**
- `fp16` enabled on GPU, disabled on CPU for compatibility
- Best checkpoint selected by `eval_loss`; adapter weights saved to `./blip2-uic-lora`


In [ ]:
OUTPUT_DIR = "./blip2-uic-lora"

# logging_dir is deprecated in favour of the TENSORBOARD_LOGGING_DIR env var
os.environ["TENSORBOARD_LOGGING_DIR"] = f"{OUTPUT_DIR}/logs"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    # ── Batch & accumulation ───────────────────────────────────────────────
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,           # effective batch = 16

    # ── Schedule ──────────────────────────────────────────────────────────
    num_train_epochs=config["num_train_epochs"],
    learning_rate=5e-4,
    warmup_steps=100,
    lr_scheduler_type="cosine",

    # ── Precision ─────────────────────────────────────────────────────────
    fp16=use_fp16,
    fp16_full_eval=use_fp16,

    # ── Evaluation & checkpointing ─────────────────────────────────────────
    eval_strategy="steps",
    eval_steps=config["eval_steps"],
    save_strategy="steps",
    save_steps=config["save_steps"],
    save_total_limit=3,                      # keep last 3 checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_steps=config["logging_steps"],
    report_to=["tensorboard"] if HAS_TENSORBOARD else [],

    # ── Misc ───────────────────────────────────────────────────────────────
    dataloader_num_workers=NUM_WORKERS,
    remove_unused_columns=False,             # keep pixel_values etc.
    label_names=["labels"],
)

# ── Generation metrics callback ────────────────────────────────────────────────
gen_metrics_callback = GenerationMetricsCallback(
    processor=processor,
    eval_records=val_eval_records,
    device=device,
    subsample=config["eval_subsample"],
)

# ── Trainer ────────────────────────────────────────────────────────────────────
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    callbacks=[gen_metrics_callback],
)

print("Trainer initialized. Starting training …")
print(f"  Epochs              : {training_args.num_train_epochs}")
print(f"  Effective batch     : {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Train records       : {len(train_dataset)}")
print(f"  Val records         : {len(val_dataset)}")
print(f"  Checkpoint dir      : {OUTPUT_DIR}")
print(f"  Logging             : {'TensorBoard' if HAS_TENSORBOARD else 'Disabled (install tensorboard)'}")

# ── TRAIN ──────────────────────────────────────────────────────────────────────
train_result = trainer.train()

print("\nTraining complete!")
print(f"  Total steps  : {train_result.global_step}")
print(f"  Train loss   : {train_result.training_loss:.4f}")


Trainer initialized. Starting training …
  Epochs              : 1
  Effective batch     : 16
  Train records       : 2540
  Val records         : 317
  Checkpoint dir      : ./blip2-uic-lora


Step,Training Loss,Validation Loss



Training complete!
  Total steps  : 159
  Train loss   : 10.6002


## 11. Post-Training Evaluation on Held-Out Test Set

Run full caption generation on the **entire test split** (no sub-sampling) and compare with the zero-shot baseline scores from `blip2_explorer_experiments.ipynb` to quantify the improvement from LoRA fine-tuning.

In [ ]:
print("Evaluating best checkpoint on full test split …")

# ── Cast all parameters to fp16 for inference (if using fp16) ─────────────────
# After training, LoRA params + biases are in fp32 but base model may be fp16.
# For consistent inference, convert everything to the appropriate dtype.
if use_fp16:
    for param in trainer.model.parameters():
        param.data = param.data.to(torch.float16)
    print("✓  Model cast to fp16 for inference")
else:
    print("✓  Model using fp32 for inference")

test_metrics = evaluate_captions(
    model=trainer.model,
    processor=processor,
    eval_records=test_eval_records,
    device=device,
    max_new_tokens=50,
    num_beams=3,
    subsample=None,        # full test set — no sub-sampling
)

print_metrics(test_metrics, label="LoRA Fine-tuned  –  Test Set")

# ── Comparison table against zero-shot baseline ───────────────────────────────
# Update these values from the blip2_explorer_experiments.ipynb results
BASELINE = {
    "Bleu_1":  None,   # fill in from experiments notebook
    "Bleu_2":  None,
    "Bleu_3":  None,
    "Bleu_4":  None,
    "CIDEr":   None,
    "METEOR":  None,
    "ROUGE_L": None,
}

print(f"\n{'─'*52}")
print(f"  {'Metric':<12} {'Zero-shot':>10}  {'LoRA FT':>10}  {'Δ':>8}")
print(f"{'─'*52}")
for k, lora_v in test_metrics.items():
    base_v = BASELINE.get(k)
    if base_v is not None:
        delta = lora_v - base_v
        sign  = "+" if delta >= 0 else ""
        print(f"  {k:<12} {base_v:>10.4f}  {lora_v:>10.4f}  {sign}{delta:>7.4f}")
    else:
        print(f"  {k:<12} {'N/A':>10}  {lora_v:>10.4f}  {'—':>8}")
print(f"{'─'*52}")
print("\n(Fill BASELINE dict above with values from blip2_explorer_experiments.ipynb)")


Evaluating best checkpoint on full test split …
✓  Model cast to fp16 for inference


Generating captions:   0%|          | 0/319 [00:00<?, ?it/s]

{'testlen': 15245, 'reflen': 4722, 'guess': [15245, 14926, 14607, 14288], 'correct': [2759, 1565, 912, 499]}
ratio: 3.2285048708167663

────────────────────────
  LoRA Fine-tuned  –  Test Set
  Metric          Score
────────────────────────
  Bleu_1         0.1810
  Bleu_2         0.1378
  Bleu_3         0.1058
  Bleu_4         0.0802
  CIDEr          0.0000
  METEOR         0.2517
  ROUGE_L        0.2694
────────────────────────


────────────────────────────────────────────────────
  Metric        Zero-shot     LoRA FT         Δ
────────────────────────────────────────────────────
  Bleu_1              N/A      0.1810         —
  Bleu_2              N/A      0.1378         —
  Bleu_3              N/A      0.1058         —
  Bleu_4              N/A      0.0802         —
  CIDEr               N/A      0.0000         —
  METEOR              N/A      0.2517         —
  ROUGE_L             N/A      0.2694         —
────────────────────────────────────────────────────

(Fill BASELINE dict 

## 12. Save LoRA Adapter Weights

`save_pretrained` on a PEFT model saves **only the LoRA delta weights** (`adapter_model.bin` / `adapter_model.safetensors` + `adapter_config.json`), not the frozen 2.7 B base parameters. The adapter can later be fused back onto any copy of `Salesforce/blip2-opt-2.7b` via `PeftModel.from_pretrained`.

In [ ]:
ADAPTER_DIR = "./blip2-uic-lora-adapter"

# ── Save LoRA adapter (delta weights only, ~few MB) ────────────────────────────
trainer.model.save_pretrained(ADAPTER_DIR)
processor.save_pretrained(ADAPTER_DIR)

# ── Persist training metadata ─────────────────────────────────────────────────
metadata = {
    "base_model":          MODEL_ID,
    "adapter_type":        "LoRA",
    "lora_r":              lora_config.r,
    "lora_alpha":          lora_config.lora_alpha,
    "lora_dropout":        lora_config.lora_dropout,
    "target_modules":      list(lora_config.target_modules),
    "bias":                lora_config.bias,
    "task_type":           str(lora_config.task_type),
    "dataset":             "UICD (Underwater Image Captioning Dataset)",
    "dataset_split":       "80/10/10",
    "captions_per_image":  5 if config["use_all_captions"] else 1,
    "num_train_records":   len(train_records),
    "num_val_records":     len(val_records),
    "num_test_records":    len(test_records),
    "num_epochs":          training_args.num_train_epochs,
    "effective_batch":     training_args.per_device_train_batch_size
                           * training_args.gradient_accumulation_steps,
    "learning_rate":       training_args.learning_rate,
    "precision":           "fp16" if use_fp16 else "fp32",
    "test_metrics":        test_metrics,
}

with open(f"{ADAPTER_DIR}/metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

# ── List saved files ──────────────────────────────────────────────────────────
adapter_files = list(Path(ADAPTER_DIR).iterdir())
print(f"\nAdapter saved to: {ADAPTER_DIR}/")
for p in sorted(adapter_files):
    size_kb = p.stat().st_size / 1024
    print(f"  {p.name:<45} {size_kb:>9.1f} KB")

# ── Reload sanity check ───────────────────────────────────────────────────────
print("\nReloading adapter to verify integrity …")
from peft import PeftModel

reloaded_base = Blip2ForConditionalGeneration.from_pretrained(
    MODEL_ID, torch_dtype=model_dtype
).to(device)
reloaded_model = PeftModel.from_pretrained(reloaded_base, ADAPTER_DIR)
reloaded_model.eval()
print("✓  Adapter reloaded successfully.")



Adapter saved to: ./blip2-uic-lora-adapter/
  README.md                                           5.1 KB
  adapter_config.json                                 1.0 KB
  adapter_model.safetensors                       13582.4 KB
  metadata.json                                       0.8 KB
  processor_config.json                               0.6 KB
  tokenizer.json                                   3475.4 KB
  tokenizer_config.json                               0.4 KB

Reloading adapter to verify integrity …


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/1247 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 14.56 GiB of which 5.81 MiB is free. Including non-PyTorch memory, this process has 14.55 GiB memory in use. Of the allocated memory 13.67 GiB is allocated by PyTorch, and 765.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)